<a href="https://colab.research.google.com/github/Curiousss/EVA/blob/master/Assignment20/Transfer_Resnet_Cifar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!ls -l
!rm test/* train/*
!rm test/cifar-100-python/* train/cifar-100-python/*
!rmdir test/cifar-100-python train/cifar-100-python
!rmdir test/ train/
!rmdir test train

In [0]:
import os
import numpy as np
import torch

import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [3]:
data_transform = transforms.Compose([transforms.Resize((197,197)), 
                                     transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
trainset = torchvision.datasets.CIFAR100(root='train/',
                                         train=True,
                                         download=True,
                                         transform=data_transform)
testset = torchvision.datasets.CIFAR100(root='test/',
                                        train=False,
                                        download=True,
                                        transform=data_transform)

Files already downloaded and verified
Files already downloaded and verified


In [0]:
# define dataloader parameters
batch_size = 128
num_workers=0

# prepare data loaders
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=True)

In [0]:
# Visualize some sample data

# obtain one batch of training images
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy() # convert images to numpy for display

# plot the images in the batch, along with the corresponding labels
#fig = plt.figure(figsize=(25, 4))
#for idx in np.arange(20):
    #ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    #plt.imshow(np.transpose(images[idx], (1, 2, 0)))
    #ax.set_title(classes[labels[idx]])

In [0]:
# Load the pretrained model from pytorch
#pretrained (bool) – If True, returns a model pre-trained on ImageNet
resnet18 = models.resnet18(pretrained=True)

# print out the model structure
#print(resnet18)

In [8]:
print(resnet18.fc.in_features) 
print(resnet18.fc.out_features) 

512
1000


In [7]:
# Freeze training for all "features" layers
#for param in resnet18.parameters():
#    param.requires_grad = False
#resnet18.avgpool.requires_grad = True
#resnet18.layer4.requires_grad_ = True
#print(resnet18)
for name, child in resnet18.named_children():
    if name in ['layer3', 'layer4']:
        print(name + 'has been unfrozen.')
        for param in child.parameters():
            param.requires_grad = True
    else:
        for param in child.parameters():
            param.requires_grad = False

layer3has been unfrozen.
layer4has been unfrozen.


In [8]:
import torch.nn as nn

n_inputs = resnet18.fc.in_features

last_layer = nn.Linear(n_inputs, 100)

resnet18.fc = last_layer

# if GPU is available, move the model to GPU
if train_on_gpu:
    resnet18.cuda()

# check to see that your last layer produces the expected number of outputs
print(resnet18.fc.out_features)
#print(resnet18)

100


In [0]:
CIFAR100_LABELS_LIST = [
    'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 
    'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 
    'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 
    'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 
    'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 
    'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion',
    'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse',
    'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear',
    'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine',
    'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose',
    'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake',
    'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table',
    'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout',
    'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman',
    'worm'
]

In [0]:
import torch.optim as optim
from torch.optim import lr_scheduler

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.001
optimizer = optim.AdamW(resnet18.parameters(), lr=0.001)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [13]:
# number of epochs to train the model
n_epochs = 5

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    
    #exp_lr_scheduler.step()

    ###################
    # train the model #
    ###################
    # model by default is set to train
    for batch_i, (data, target) in enumerate(train_loader):
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = resnet18(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss 
        train_loss += loss.item()
        
    exp_lr_scheduler.step()
    print('Epoch %d, Batch %d loss: %.16f' %(epoch, batch_i + 1, train_loss / (batch_i)))
    train_loss = 0.0

Epoch 1, Batch 391 loss: 0.7019589031353975
Epoch 2, Batch 391 loss: 0.4268446434766818
Epoch 3, Batch 391 loss: 0.1232785094147309
Epoch 4, Batch 391 loss: 0.0656632228444020
Epoch 5, Batch 391 loss: 0.0457607636467004


In [14]:
test_loss = 0.0
class_correct = list(0. for i in range(100))
class_total = list(0. for i in range(100))

resnet18.eval() # eval mode

# iterate over test data
for data, target in test_loader:
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = resnet18(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update  test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    i = 0
    for t in target:
        label = t.data
        class_correct[label] += correct[i].item()
        class_total[label] += 1
        i = i + 1

# calculate avg test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(100):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            CIFAR100_LABELS_LIST[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5d: N/A (no training examples)' % i)

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 1.174624

Test Accuracy of apple: 89% (89/100)
Test Accuracy of aquarium_fish: 88% (88/100)
Test Accuracy of  baby: 66% (66/100)
Test Accuracy of  bear: 60% (60/100)
Test Accuracy of beaver: 56% (56/100)
Test Accuracy of   bed: 72% (72/100)
Test Accuracy of   bee: 74% (74/100)
Test Accuracy of beetle: 83% (83/100)
Test Accuracy of bicycle: 85% (85/100)
Test Accuracy of bottle: 91% (91/100)
Test Accuracy of  bowl: 69% (69/100)
Test Accuracy of   boy: 72% (72/100)
Test Accuracy of bridge: 79% (79/100)
Test Accuracy of   bus: 69% (69/100)
Test Accuracy of butterfly: 72% (72/100)
Test Accuracy of camel: 73% (73/100)
Test Accuracy of   can: 83% (83/100)
Test Accuracy of castle: 85% (85/100)
Test Accuracy of caterpillar: 80% (80/100)
Test Accuracy of cattle: 79% (79/100)
Test Accuracy of chair: 84% (84/100)
Test Accuracy of chimpanzee: 83% (83/100)
Test Accuracy of clock: 84% (84/100)
Test Accuracy of cloud: 84% (84/100)
Test Accuracy of cockroach: 76% (76/100)
Test Accuracy of co

In [0]:
# obtain one batch of test images
dataiter = iter(test_loader)
images, labels = dataiter.next()
#images.numpy()

# move model inputs to cuda, if GPU available
if train_on_gpu:
    images = images.cuda()

# get sample outputs
output = resnet18(images)
# convert output probabilities to predicted class
_, preds_tensor = torch.max(output, 1)
preds = np.squeeze(preds_tensor.numpy()) if not train_on_gpu else np.squeeze(preds_tensor.cpu().numpy())
images = images.cpu()

# plot the images in the batch, along with predicted and true labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    #print(images[idx])
    plt.imshow(np.transpose(images[idx], (1,2,0)))
    ax.set_title("{} ({})".format(CIFAR100_LABELS_LIST[preds[idx]], CIFAR100_LABELS_LIST[labels[idx]]),
                 color=("green" if preds[idx]==labels[idx].item() else "red"))